# Perfomed this experiment on Google Colab Pro version using compute instance.
GPU: Tesla V100-SXM2-16GB <br>
Dataset: Soybean Disease Dataset: https://www.kaggle.com/datasets/shuvoalok98/soybean-disease-dataset <br>
Model is trained on images from single category i.e Caterpillar for 1000 kimg 


In [7]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
  steps followed for doing the setup.
  1. Install anaconda or miniconda.
  2. Create new environment and activate it.
  3. Install python version 3.7 as needed for StyleGAN2-ADA model.
  4. Install all the compatible libraries within activate conda environment either using pip or conda.
    or specify all the packages within requirements.txt file and run using pip command.

Note:- 1. All the below specified commands are done through command line within Google Colab Pro compute instance.
       2. For the versions comptability please refer the original github repo i.e https://github.com/NVlabs/stylegan2-ada-pytorch/tree/main
'''

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh
!conda create -n image_gen
!conda activate image_gen
!conda install python=3.7
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3
  # or Mention all the dependencies in requirements.txt file
!pip install -r ./requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

# Note: You may also need to install packages: psutil and scipy

In [8]:
''' Credit to Authors Tero Karras, Miika Aittala, Janne Hellsten, Samuli Laine, Jaakko Lehtinen,
    Timo Aila of Training Generative Adversarial Networks with Limited Data'''

# clone the git repo
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 7.76 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [10]:
!cd /content/stylegan2-ada-pytorch

In [ ]:
''' Before training/finetuning the model transform the dataset to 256x256 resolution to make it of equal dimensions.
    Also here I am trying to lower the dimension to fasten up the training process.
    original resoultion is 500 x 500 '''

# Here we are zipping the dataset file as needed in this format for the model.
!python dataset_tool.py --source=../drive/MyDrive/raw_data/data/soybean_data/Caterpillar --dest=./datasets/caterpillar.zip --width=256 --height=256

In [ ]:
# Training the model on custom dataset

!python train.py --outdir=./training-runs --data=./datasets/caterpillar.zip --gpus=1 --kimg=1000

In [ ]:
# In case if you want to resume the training from certain checkpoint.
# Please update the pkl file name accordingly depending on the checkpoint you want to resume.

!python train.py --outdir=./training-runs --data=./datasets/caterpillar.zip --gpus=1 --kimg=1000 —-resume=./training-runs/00001-caterpillar-auto1-kimg1000/network-snapshot-000200.pkl

In [ ]:
''' Generating images for random seeds using pretrained model on soybean leaf images.
    The model has been trained for 1000 kimg, hence used the last snapshot for generating random seeds.
'''

!python3 generate.py --outdir=out_dir --trunc=1 --seeds=85,265,297,849 --network=./training-runs/00004-caterpillar-auto1-kimg1000/network-snapshot-001000.pkl

In [ ]:
# Generating list of images using trained model on soybean leaf images.

!python3 generate.py --outdir=out_dir --trunc=1 --seeds=0-2000 --network=./training-runs/00004-caterpillar-auto1-kimg1000/network-snapshot-001000.pkl

In [ ]:
'''
  After generating the synthetic images use Laplacian filter to remove the blurry ones.
  Code for Laplacian filter taken from the github repo https://github.com/yunusa2k2/GANLapRice.

  Credit to Authors Yunusa Haruna, Shiyin Qin and Mesmin J. Mbyamm Kiki for the inspiration.
'''
!python Laplacian.py --images==./out/